<a href="https://colab.research.google.com/github/Multi-Objective-NAS/invertible-nas-algorithm/blob/master/GDE%2BNasBench101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [0]:
!git clone https://github.com/Zymrael/gde
!git submodule init
!git submodule update

Cloning into 'gde'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 67 (delta 29), reused 29 (delta 6), pack-reused 0
Unpacking objects: 100% (67/67), done.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [0]:
!pip install dgl-cu101

     |████████████████████████████████| 16.2MB 201kB/s 


In [0]:
!pip install git+https://github.com/rtqichen/torchdiffeq

  Cloning https://github.com/rtqichen/torchdiffeq to /tmp/pip-req-build-2pdj14i9
  Running command git clone -q https://github.com/rtqichen/torchdiffeq /tmp/pip-req-build-2pdj14i9
  Created wheel for torchdiffeq: filename=torchdiffeq-0.0.1-cp36-none-any.whl size=27578 sha256=f951d56d2362baaa9591c790e332789967cabdaa8882787e80e3b974212227a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-gjgg2xic/wheels/3f/76/69/01867bf3355c3bc8bae7e556b17b44c395b6cda5e76fd8ddc7
Successfully built torchdiffeq


#Installation dataset

In [0]:
#!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
!pip intall ./nasbench

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0  74.7M      0  0:00:06  0:00:06 --:--:-- 65.4M
Cloning into 'nasbench'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 96 (delta 0), reused 2 (delta 0), pack-reused 92
Unpacking objects: 100% (96/96), done.
ERROR: unknown command "intall" - maybe you meant "install"


#Setting

### Import Libraries

In [0]:
import math
import numpy as np
import scipy.sparse as sp
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from gde.torchgde import PerformanceContainer, accuracy

In [0]:
import dgl
import dgl.data
import dgl.function as fn
import networkx as nx
from dgl import DGLGraph
from dgl.data.utils import get_download_dir

### Constants

In [0]:
DEVICE = torch.device('cuda:0')
#DEVICE = torch.device('cpu')
BATCH_SIZE = 50
LR = 0.1
NEPOCHS = 1 
RTOL = 1e-7
ATOL = 1e-9
EDGE_NUM = 9

INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'
NOOPT = 'noopt'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_LABEL = [NOOPT, CONV3X3, CONV1X1, MAXPOOL3X3, OUTPUT]

#Code

### Data 

In [0]:
from nasbench import api

nasbench_api = api.NASBench('nasbench_only108.tfrecord')

/content/gde/nasbench


ImportError: ignored

In [0]:
class NAGRAPH(object):

    def __init__(self, mat, op):
        self.mat = mat
        self.op = op
        
        

In [0]:
"""
1.  network architecture -> DGLG object
2.  node -> feature vector
3.  input = (DGLG, feature matrix), output = [accuracy, latency]
"""
class DataLoader(object):
    def __init__(self, start, end, batchsize=None):
        self.start = start
        self.end = end
        self.cur = start
        if batchsize is None:
            self.batchsize = end - start
        else:
            self.batchsize = batchsize
        self.size = end - start
        self. nasbench_hashkeys = nasbench_api.hash_iterator()

    def __run_experiment(self, mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench_api.query(model_spec)
        return q['validation_accuracy'], q['training_time']

    def next(self):
        global nasbench_hashkeys

        xset = []
        yset = []
        if self.cur == self.end:
            self.cur = self.end
        for idx in range(self.cur, min(self.cur + self.batchsize, self.end)):
            hash_val = nasbench_hashkeys[idx]
            fixed_stat, _ = nasbench_api.get_metrics_from_hash(hash_val)
            mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
            x = encode(mat, op)
            y = self.__run_experiment(mat, op)
            xset.append(x)
            yset.append(y)
        xset = torch.FloatTensor(xset)
        yset = torch.FloatTensor(yset)
        return xset, yset


def train_val_test_split(batch_size=100, validation_ratio=0.1, test_ratio=0.1):
    global nasbench_hashkeys

    revised = []
    for hash_val in nasbench_hashkeys:
        fixed_stat, _ = nasbench_api.get_metrics_from_hash(hash_val)
        mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
        if np.sum(mat) == EDGE_NUM:
            revised.append(hash_val)

    total_size = (len(revised) // batch_size) * batch_size
    nasbench_hashkeys = revised[: total_size]
    val_size = int(total_size * validation_ratio)
    test_size = int(total_size * test_ratio)

    return DataLoader(val_size+test_size, total_size, batch_size), DataLoader(0, val_size), DataLoader(val_size, val_size+test_size)

In [0]:
'''
node feature = 
(node ordering up to 7) + (edge label) * 9
'''

def encode(mat, ops):
    def _sanity_check(idx1, opidx, idx2):
        if idx1 not in range(7):
            return False
        if idx2 not in range(7):
            return False
        if opidx not in range(4):
            return False
        return True

    encoded = []
    for inbound, outbound in zip(*np.array(mat).nonzero()):
        op = ALLOWED_OPS.index(ops[outbound])
        assert _sanity_check(inbound, op, outbound)

        embed = [0] * EMBED_SIZE
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1

        encoded.append(embed)

    assert np.array(encoded).shape == (9 , EMBED_SIZE)
    return encoded


def max_index(output):
    # (1, DIM, 18,18)
    encoded = []
    outputs = [output_ for output_ in output[0]]
    
    for output in outputs:
        embed = [0 for _ in range(EMBED_SIZE)]
        inbound, op, outbound = np.argmax(output[:7]), np.argmax(output[7:-7]), np.argmax(output[-7:])
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1
        assert np.sum(np.array(embed)) == 3
        encoded.append(embed)

    assert np.array(encoded).shape == (9, EMBED_SIZE)
    return encoded


def decode(encoded):
    mat = np.zeros((7, 7))
    ops = [INPUT] + [CONV3X3 for _ in range(5)] + [OUTPUT]

    for embed in encoded:
        inbound, op, outbound = np.nonzero(np.array(embed))[0]
        op -= 7
        outbound -= (7 + len(ALLOWED_OPS))
        ops[outbound] = ALLOWED_OPS[op]
        if inbound > outbound:
            (inbound, outbound) = (outbound, inbound)
        elif inbound == outbound:
            continue
        mat[inbound][outbound] = 1

    return mat.tolist(), ops

### Model Architecture Module

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm( dim, dim)

In [0]:
class ConcatConv1d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(ConcatConv1d, self).__init__()
        self._layer = nn.Conv1d(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out

In [0]:
#test
a = torch.ones(3, 2)
b = torch.zeros(3, 3)
torch.cat([a, b], dim=1)

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

In [0]:
class ODEBlock(nn.Module):

    def __init__(self, odefunc, method):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()
        self.method = method

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        #batch_t = T_series[:x.shape[0]]
        out = odeint(self.odefunc, x, self.integration_time , rtol=RTOL, atol=ATOL, method = self.method)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value

In [0]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        ret = x.view(-1, shape)
        return ret


### Optional Functions

In [0]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


In [0]:
def learning_rate_with_decay(batch_size, batch_denom, batches_per_epoch, boundary_epochs, decay_rates):
    initial_learning_rate = LR * batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn

In [0]:
def accuracy(model, dataloader):
    datasize = dataloader.end - dataloader.start
    x, y = dataloader.next(datasize)
    x = torch.FloatTensor(x)
    x = x.to(DEVICE)
    y = y.numpy()
    model._modules['1'].odefunc.nfe = 0
    result = model(x).cpu().detach().numpy()
    total_correct = np.sum(np.square(result - y))
    return total_correct / datasize

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Model

In [8]:
# Test

def add(parameters):
    a = parameters['a']
    b = parameters['b']
    if 'c' in parameters:
        c = parameters['c']
    return a+b

def passing():
    return {'a': 1, 'b': 2}

out = passing()
print(add(out))

3


In [0]:
class GCNLayer(nn.Module):
    def __init__(self,in_feats:int, out_feats:int, activation:Callable[[torch.Tensor], torch.Tensor],
                 dropout:int, bias:bool=True):
        super().__init__()
        self.g = None
        self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.bias = None
        self.activation = activation
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, inputs):
        g, h = inputs
        if not g is None:
            self.g = g
        if self.dropout:
            h = self.dropout(h)
        h = torch.mm(h, self.weight)
        # normalization by square root of src degree
        h = h * ndata['norm']
        self.g.ndata['h'] = h
        self.g.update_all(fn.copy_src(src='h', out='m'),
                          fn.sum(msg='m', out='h'))
        h = self.g.ndata.pop('h')
        # normalization by square root of dst degree
        h = h * ndata['norm']
        # bias
        if self.bias is not None:
            h = h + self.bias
        if self.activation:
            h = self.activation(h)
        return g, h

In [0]:
class GDEFunc(nn.Module):
    def __init__(self, gnn:nn.Module):
        """General GDE function class. To be passed to an ODEBlock"""
        super().__init__()
        self.gnn = gnn
        self.nfe = 0
    
    def set_graph(self, g:dgl.DGLGraph):
        for layer in self.gnn:
            layer.g = g
            
    def forward(self, t, h):
        self.nfe += 1
        , out = self.gnn((None, h))
        return out

In [0]:
class ODEBlock(nn.Module):
    def __init__(self, odefunc:nn.Module, method:str='dopri5', rtol:float=1e-3, atol:float=1e-4, adjoint:bool=True):
        """ Standard ODEBlock class. Can handle all types of ODE functions
            :method:str = {'euler', 'rk4', 'dopri5', 'adams'}
        """
        super().__init__()
        self.odefunc = odefunc
        self.method = method
        self.adjoint_flag = adjoint
        self.atol, self.rtol = atol, rtol

    def forward(self, inputs, T:int=1):
        g, h = inputs

        self.integration_time = torch.tensor([0, T]).float()
        self.integration_time = self.integration_time.type_as(h)
        
        odefunc.set_graph(g)

        if self.adjoint_flag:
            out = torchdiffeq.odeint_adjoint(self.odefunc, h, self.integration_time,
                                             rtol=self.rtol, atol=self.atol, method=self.method)
        else:
            out = torchdiffeq.odeint(self.odefunc, h, self.integration_time,
                                     rtol=self.rtol, atol=self.atol, method=self.method)
            
        return g, out[-1]
    
    def forward_batched(self, inputs, nn:int, indices:list, timestamps:set):
        """ Modified forward for ODE batches with different integration times """
        g, h = inputs

        timestamps = torch.Tensor(list(timestamps))

        odefunc.set_graph(g)

        if self.adjoint_flag:
            out = torchdiffeq.odeint_adjoint(self.odefunc, h, timestamps,
                                             rtol=self.rtol, atol=self.atol, method=self.method)
        else:
            out = torchdiffeq.odeint(self.odefunc, h, timestamps,
                                     rtol=self.rtol, atol=self.atol, method=self.method)

        out = self._build_batch(out, nn, indices).reshape(h.shape)
        return g, out
    
    def _build_batch(self, odeout, nn, indices):
        b_out = []
        for i in range(len(indices)):
            b_out.append(odeout[indices[i],i*nn:(i+1)*nn])
        return torch.cat(b_out).to(odeout.device)
              
        
    def trajectory(self, x:torch.Tensor, T:int, num_points:int):
        self.integration_time = torch.linspace(0, t_end, num_points)
        self.integration_time = self.integration_time.type_as(x)
        out = torchdiffeq.odeint(self.odefunc, x, self.integration_time,
                                 rtol=self.rtol, atol=self.atol, method=self.method)
        return out

In [0]:
class ODEModel(nn.Module):
    def __init__(self, num_feats, n_classes):
        # dim is edge number
        super(ODEModel, self).__init__()
        gnn = nn.Sequential(GCNLayer(in_feats=64, out_feats=64, activation=nn.Softplus(), dropout=0.9),
                  GCNLayer(in_feats=64, out_feats=64, activation=None, dropout=0.9)
                 ).to(DEVICE)

        gdefunc = GDEFunc(gnn)
        gde = ODEBlock(odefunc=gdefunc, method='rk4', atol=1e-3, rtol=1e-4, adjoint=False).to(DEVICE)
        self.model = nn.Sequential(GCNLayer(in_feats=num_feats, out_feats=64, activation=F.relu, dropout=0.4),
                  gde,
                  GCNLayer(in_feats=64, out_feats=n_classes, activation=None, dropout=0.)
                  ).to(DEVICE)

    def forward(self, inputs):
        g, h = inputs
        return self.model((g, h)))[1]

### Trainer

In [0]:
class Trainer():
    def __init__(self, model, optimizer, criterion, accuracy, lr_fn, meter_fn):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.accuracy = accuracy
        self.lr_fn = lr_fn
        self.batch_time_meter = meter_fn
        self.f_nfe_meter = meter_fn
        self.b_nfe_meter = meter_fn
    
    def run(self, epochs, train_dataloader, val_dataloader, test_dataloader):
        train_size = train_dataloader.size
        batches_per_eoch = train_size // train_dataloader.batchsize
        for itr in range(epochs):
            for batch_turn in range(batches_per_epoch):
                x, y = train_dataloader.next()

                nfe = self.model._modules['1'].odefunc.nfe

                x = x.to(DEVICE)
                y = y.to(DEVICE)
                self.model.train()
                logits = self.model(x)

                result = logits.cpu().detach().numpy()
                loss = self.criterion(logits, y)
                self.optimizer.zero_grad()
                
                loss.backward()

                self.optimizer.step()

                print("[%d] Completed one batch" % batch_turn)

                #with train_summary_writer.as_default():
                    #tensorflow.summary.scalar('loss', loss.item(), step=globaliter)

            with torch.no_grad():
                self.model.eval()
                train_acc = self.accuracy(self.model, test_dataloader)
                val_acc = self.accuracy(self.model, val_dataloader)
                print(
                     "Epoch {:04d} | Time {:.3f} ({:.3f}) | NFE-F {:.1f} | NFE-B {:.1f} | Train Acc {:.4f} | Test Acc {:.4f}".format( itr // batches_per_epoch, batch_time_meter.val, batch_time_meter.avg, f_nfe_meter.avg, b_nfe_meter.avg, train_acc, val_acc )
                     )

#Data Generation

In [0]:
# seed for repeatability
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.manual_seed(0)
np.random.seed(0)

train_dataloader, test_dataloader, val_dataloader = train_val_test_split(batch_size=BATCH_SIZE, validation_ratio=0.1, test_ratio=0.1)

#Initialization

In [0]:
model = ODEModel() #method is None
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.MSELoss().to(DEVICE)
batches_per_epoch = train_dataloader.size // train_dataloader.batchsize
lr_fn = learning_rate_with_decay(
        BATCH_SIZE, batch_denom=128, batches_per_epoch=batches_per_epoch, boundary_epochs=[60, 100, 140],
        decay_rates=[1, 0.1, 0.01, 0.001]
)
trainer = Trainer(model=model,optimizer=optimizer,criterion=criterion,
                  accuracy=accuracy,lr_fn=lr_fn,meter_fn=RunningAverageMeter())

# Train

In [0]:
#%tensorboard --logdir logs/tensorboard
trainer.run(NEPOCHS, train_dataloader, val_dataloader, test_dataloader)

[0] Completed one batch
[1] Completed one batch


# Predict input

adjoint.py의 81번째 코드 참고!

In [0]:
# predict x by y
search_size = 1
for itr in range(min(train_size, search_size)):
    x,y = train_dataloader.next(1)
    y = np.transpose(y.numpy())

    # Linear
    weight = model[5].weight
    bias = model[5].bias.cpu().detach().numpy()
    bias = bias.reshape((2,1))
    z_tn = np.subtract(y, bias)
    inverse_weight = np.linalg.pinv(weight.cpu().detach().numpy())
    z_tn = np.matmul(inverse_weight, z_tn)

    # AdaptiveAvgPool1d(1)
    z_tn = np.repeat(z_tn, EMBED_SIZE, axis=1)
    z_tn = np.expand_dims(z_tn, axis=0)

    # ode back
    z_tn = torch.from_numpy(z_tn).to(DEVICE)
    z_t0 = odeint(ODEf, z_tn, torch.tensor([1, 0]).float().type_as(z_tn) , rtol=RTOL, atol=ATOL, method = 'adams')[1]
    # output[0] is for time t0, output[1] is for time t1.

    assert z_t0.shape == (1, 9,18)

    z_t0 = z_t0.cpu().detach().numpy()
    encoded = max_index(z_t0)
    
    print("encoded")
    for enc_ in encoded:
        print(enc_)
    
    mat, op = decode(encoded)

    print("mat")
    for mat_ in mat:
        print(mat_)
    print("y:", y)


    def __run_experiment(mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench_api.query(model_spec)
        return q['validation_accuracy'], q['training_time']


    print("result", __run_experiment(mat, op))